# <span style="color:darkblue">**Forecasting the S\&P 500**</span>

In [ ]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings('ignore')

#### <span style="color:red">**Load and Shape - Raw Data**</span>

##### <span style="color:green">Things to Do: Load and Shape - Raw Data</span>
1. Convert index in the pandas Series into an appropriate datetime index **(make date slicing easy and intuitive)**
2. Create a second pandas Series that samples at calendar month end dates **(test prediction quality using monthly frequency)**
3. Create a third pandas Series that samples at calender quarter end dates **(test prediction quality using quarterly frequency)**

In [ ]:
data = pd.read_csv("./sp500.csv", index_col=0, parse_dates=True)

In [ ]:
data = data.close.dropna()

#### <span style="color:red">**Exploratory Data Analysis - Raw Data**</span>

##### <span style="color:green">Things to Do: Exploratory Data Analysis - Raw Data</span>
1. Add EDA features from Chapters 1 & 2 *Stastical Thinking in Python (Part 1)* **(expand practical application of python and data science skills)**
2. Translate insights from full EDA into prediction methodology **(intelligently create a system to apply various forecasting tools)**

In [ ]:
fig, ax = plt.subplots()
data.plot(ax=ax, figsize=(12,8))
plt.show()

#### <span style="color:red">**Test Stationarity - Raw Data**</span>

In [ ]:
adf = adfuller(data)

#### <span style="color:red">**Transform Raw Data to Achieve Stationarity**</span>

##### <span style="color:green">Things to Do: Transform Raw Data to Achieve Stationarity</span>
1. Understand how to inverse transform in order to generate predictions on the variable of interest **(required step in a prediction pipeline, allows you do apply the best inverse transformation process - i.e. best non-stationary properties)**

In [ ]:
diff = data.diff(1).dropna()
pct = (data.shift(1)/data).dropna()
log = np.log(data/data.shift(1)).dropna()

#### <span style="color:red">**Exploratory Data Analysis - Transformed Data**</span>

In [ ]:
fig, ax = plt.subplots()
diff.plot(ax=ax, figsize=(12,8))
plt.show()

In [ ]:
fig, ax = plt.subplots()
pct.plot(ax=ax, figsize=(12,8))
plt.show()

In [ ]:
fig, ax = plt.subplots()
log.plot(ax=ax, figsize=(12,8))
plt.show()

In [ ]:
diff_adf = adfuller(diff)
pct_adf = adfuller(pct)
log_adf = adfuller(log)

In [ ]:
diff_adf

In [ ]:
pct_adf

In [ ]:
log_adf

## <span style="color:red">**SARIMAX Model on a Perfect Sinusoidal Time Series**</span>

In [ ]:
perfect = pd.read_csv("./perfect_cyclical_ts.csv", index_col=0, parse_dates=True)
perfect = perfect.level.dropna()
perfect_adf = adfuller(perfect)
perfect.plot()

In [ ]:
diff_perfect = perfect.diff().dropna()
diff_perfect_adf = adfuller(diff_perfect)
diff_perfect.plot()

In [ ]:
model_order = (6, 0, 0)
arima_model_order = (6, 1, 0)

arma_diff_perfect = SARIMAX(diff_perfect, order=model_order)
fit_arma_diff_perfect = arma_diff_perfect.fit()

isp_diff_perfect = fit_arma_diff_perfect.get_prediction(start=-365, dynamic=True)
isp_diff_perfect_mean = isp_diff_perfect.predicted_mean
isp_ci = isp_diff_perfect.conf_int()
isp_lci = isp_ci.loc[:,"lower level"]
isp_uci = isp_ci.loc[:,"upper level"]

_ = plt.figure(figsize=(15,5))
_ = plt.plot(diff_perfect.index[-1460:],  diff_perfect[-1460:], label="observed")
_ = plt.plot(isp_diff_perfect_mean.index, isp_diff_perfect_mean.values, color="r", label="dynamic prediction")
_ = plt.fill_between(isp_lci.index, isp_lci, isp_uci, color="pink")
_ = plt.xlabel("Date")
_ = plt.ylabel("Level - First Order Difference")
_ = plt.legend(loc="upper left")
plt.show()

In [ ]:
isp_perfect_invt = np.cumsum(isp_diff_perfect_mean)
isp_lci_invt = np.cumsum(isp_ci.loc[:,"lower level"])
isp_uci_invt = np.cumsum(isp_ci.loc[:,"upper level"])

isp_perfect = perfect.iloc[-365] + isp_perfect_invt
isp_perfect_lci = perfect.iloc[-365] + isp_lci_invt
isp_perfect_uci = perfect.iloc[-365] + isp_uci_invt

_ = plt.figure(figsize=(15,5))
_ = plt.plot(perfect.index[-1460:],  perfect[-1460:], label="observed")
_ = plt.plot(isp_perfect.index, isp_perfect.values, color="r", label="dynamic prediction")
_ = plt.fill_between(isp_perfect_lci.index, isp_perfect_lci, isp_perfect_uci, color="pink")
_ = plt.xlabel("Date")
_ = plt.ylabel("Level")
_ = plt.legend(loc="upper left")
plt.show()

In [ ]:
arima_perfect = SARIMAX(perfect, order=arima_model_order)    #, initialization='approximate_diffuse')
fit_arima_perfect = arima_perfect.fit()
fit_arima_perfect.summary()

isp_arima_perfect = fit_arima_perfect.get_prediction(start=-365, dynamic=True)
isp_arima_perfect_mean = isp_arima_perfect.predicted_mean
isp_arima_perfect_ci = isp_arima_perfect.conf_int()
isp_arima_perfect_lci = isp_arima_perfect_ci.loc[:,"lower level"]
isp_arima_perfect_uci = isp_arima_perfect_ci.loc[:,"upper level"]

_ = plt.figure(figsize=(15,5))
_ = plt.plot(perfect.index[-1460:],  perfect[-1460:], label="observed")
_ = plt.plot(isp_arima_perfect_mean.index, isp_arima_perfect_mean.values, color="r", label="dynamic prediction")
_ = plt.fill_between(isp_arima_perfect_lci.index, isp_arima_perfect_lci, isp_arima_perfect_uci, color="pink")
_ = plt.xlabel("Date")
_ = plt.ylabel("Level")
_ = plt.legend(loc="upper left")
plt.show()

#### <span style="color:darkred">SARIMAX Model - Baseline ARMA Subclass Model</span>

##### <span style="color:darkgreen">Things to Do: SARIMAX Model - ARMA Subclass</span>
1. Transform the raw data - for now just calculate the first order difference **(most financial data is going to be non-stationary)**
2. Fit the ARMA subclass model to the first order difference
3. Use dynamic mode to forecast the last 50 days of in-sample training data
4. Use forecast model to forecast the last 50 days of out-of-sample test data using previous data as the training set

In [ ]:
arma_diff = SARIMAX(diff, order=(2, 0, 2), trend="c")
fit_diff = arma_diff.fit()

In [ ]:
fit_diff.summary()

In [ ]:
isf = fit_diff.get_prediction(start=-50, dynamic=True)
isf_meanp = isf.predicted_mean
isf_ci = isf.conf_int()
isf_lci = isf_ci.loc[:,"lower close"]
isf_uci = isf_ci.loc[:,"upper close"]
isf_meanp

In [ ]:
_ = plt.figure()
_ = plt.plot(diff.index[-50:],  diff[-50:], label="observed")
_ = plt.plot(isf_meanp.index, isf_meanp.values, color="r", label="dynamic prediction")
plt.show()

In [ ]:
isf_intp = np.cumsum(isf_meanp)
close_forecast = osf_int_forecast + close
close_forecast[-50:]

In [ ]:
close_arima = SARIMAX(close, order=(2, 1, 2))
close_arima_fit = close_arima.fit()

In [ ]:
osf_arima = close_arima_fit.get_prediction(start=-30)
osf_close_forecast = osf_arima.predicted_mean
osf_close_forecast

## Generating ARMA Data (Appendix)

In [ ]:
from statsmodels.tsa.arima_process import arma_generate_sample
np.random.seed(3)

# set the ARMA model coefficients
ar_coefs = [1, 0.2]
ma_coefs = [1, 0.3, 0.4]

# generate time-series data
y = arma_generate_sample(ar_coefs, ma_coefs, nsample=4000, sigma=0.5)

# plot the time-series
plt.plot(y)
plt.ylabel(r"$y-t$")
plt.xlabel(r"$t$")
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARMA
model = ARMA(y, order=(1,2))
results = model.fit()
results.summary()